# Jupyter notebook for the course project

In this notebook, I will exemplify the use of some functions and modules. 

The modules were developed as a tool to extract data from the outputs created by VASP (Vienna Ab-initio Simulation Package), and to create a database in the ASE format (Atomic Simulation Environment, see https://wiki.fysik.dtu.dk/ase/ase/db/db.html ).

The code follows the PEP8 for syntax and PEP257 for documentation.

### Modules and functions

Stored in the folder named "MyPythonLibs"

The required python modules are:
* numpy
* ase
* pymatgen

### Required modules to import

In [1]:
import sys
import os
import re
import platform
import notebook
import ase
import ase.db
import numpy as np

### Specific functions

In [2]:
from ase import db
from ageo import dbtools, vasptools 

In [3]:
help(dbtools)

Help on module ageo.dbtools in ageo:

NAME
    ageo.dbtools

FILE
    /home/ageo/MyPythonLibs/ageo/dbtools.py

FUNCTIONS
    add_vasp_structure(dbname, folder)
        Add a VASP structure into a database.
        
        Keyword arguments:
        dbname -- database filename
        folder -- path with VASP outputs
    
    db_read_row(dbname, id)
        Reads a specific row in a ASE database.
        
        Keyword arguments:
        dbname -- database name.
        id -- id number of the row to read.




In [4]:
help(vasptools)

Help on module ageo.vasptools in ageo:

NAME
    ageo.vasptools

FILE
    /home/ageo/MyPythonLibs/ageo/vasptools.py

FUNCTIONS
    d_center(slab_xml, vac_pot_slab, fermi_slab)
        Calculate the d-band center of a structure
        
        Keyword arguments:
        slab_xml -- parsed vasprun.xml to be analysed.
        vac_pot_slab -- vacuum potential of the slab, or the Fermi energy of the
        slab.
        fermi_slab -- fermi energy extracted from vasprun.xml.
    
    e_ads(n_ads, et_surf_ads, et_surf, et_ads)
        "Calculate the adsorption energy per atom.
        
        Keyword arguments:
        n_ads -- number of adsorbates in the supercell.
        et_surf_ads -- total energy of the system containing the surface and
        the adsorbate.
        et_surf -- total energy of the system containing the clean slab.
        et_ads -- total energy of one atom/molecule of the adsorbate.
    
    e_coh(e_bulk, e_atom, n_atom_bulk)
        Calculate the cohesive energy of a

### Creating an ASE database

In [5]:
# The database name
dbname= "project.db"

# Make sure always have a new database when the first element is added
try:
    os.remove(dbname)
except:
    print()

### Add VASP structure to the database

Note that you have to specify the full path in order to locate this file in yor machine

In [6]:
# Isolated Ni atom
row_id = dbtools.add_vasp_structure(dbname, "/home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_atom/")

('Structure added with id:', 1)


In [7]:
# Ni bulk
row_id = dbtools.add_vasp_structure(dbname, "/home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_bulk/")

# Ni slab 100
row_id = dbtools.add_vasp_structure(dbname, "/home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_slab_100/")

('Structure added with id:', 2)
('Structure added with id:', 3)


### Reading the atoms object from the database

In [8]:
Ni_atom = dbtools.db_read_row(dbname,1)
Ni_bulk = dbtools.db_read_row(dbname,2)
Ni_slab_100 = dbtools.db_read_row(dbname,3)

### Reading VASP xml files
Note that you have to specify the full path in order to locate this file in yor machine

In [9]:
Ni_atom_xml = vasptools.read_vasprun("/home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_atom/vasprun.xml")
Ni_bulk_xml = vasptools.read_vasprun("/home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_bulk/vasprun.xml")
Ni_slab_100_xml = vasptools.read_vasprun("/home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_slab_100/vasprun.xml")

/home/ageo/.local/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


/home/ageo/.local/lib/python2.7/site-packages/pymatgen/io/vasp/outputs.py:154: UserWarning: Float overflow (*******) encountered in vasprun
  warnings.warn('Float overflow (*******) encountered in vasprun')
/home/ageo/.local/lib/python2.7/site-packages/pymatgen/io/vasp/outputs.py:877: UserWarning: No POTCAR file with matching TITEL fields was found in /home/ageo/Documents/PhD_Documents/Courses/Adv_Sci_Python/Project/new_project/examples/Ni_slab_100
  " was found in {}".format(os.path.abspath(p)))


### Using ASE database and atoms object to extract information for further analisys

#### Total energy

In [10]:
Ni_atom_E = Ni_atom.get_total_energy()

Ni_bulk_E = Ni_bulk.get_total_energy()

Ni_slab_100_E = Ni_slab_100.get_total_energy()

#### Number of atoms

In [11]:
Ni_atom_n = Ni_atom.get_number_of_atoms()

Ni_bulk_n = Ni_bulk.get_number_of_atoms()

Ni_slab_100_n = Ni_slab_100.get_number_of_atoms()

### Calculating some properties using a combination of pymatgen and my own module "vasptools"

#### Bulk cohesive energy

In [12]:
help(vasptools.e_coh)

Help on function e_coh in module ageo.vasptools:

e_coh(e_bulk, e_atom, n_atom_bulk)
    Calculate the cohesive energy of a bulk material.
    
    Keyword arguments:
    e_bulk -- total energy of the bulk structure.
    e_atom -- total energy of a isolated atom.
    n_atom_bulk -- number of atoms in the bulk.



In [13]:
Ni_bulk_e_coh = vasptools.e_coh(Ni_bulk_E, Ni_atom_E, Ni_bulk_n)

#### Surface area

In [14]:
Ni_slab_100_A  = Ni_slab_100.get_volume() / Ni_slab_100.get_cell()[2][2] 

#### Surface formation energy

In [15]:
help(vasptools.e_surf_form)

Help on function e_surf_form in module ageo.vasptools:

e_surf_form(e_t_surf, e_t_bulk, n_atoms_surf, n_atoms_bulk, area)
    Calculate the surface formation energy per area of the surface.
    
    Keyword arguments:
    e_t_surf -- total energy of the surface slab.
    e_t_tbulk -- total energy of the bulk.
    n_atoms_surf -- number fo atoms in the slab unit cell.
    n_atoms bulk -- number of atoms in a bulk unit cell.
    area -- area of the slab.



In [16]:
Ni_slab_100_Ef = vasptools.e_surf_form(Ni_slab_100_E, Ni_bulk_E, Ni_slab_100_n, Ni_bulk_n, Ni_slab_100_A)
print(Ni_slab_100_Ef)

0.1376240093166087


#### Fermi energy

In [17]:
# Using pymatgen to extract from the vasprun.xml file
Ni_atom_Efermi = Ni_atom_xml.efermi

Ni_bulk_Efermi = Ni_bulk_xml.efermi

Ni_slab_100_Efermi = Ni_slab_100_xml.efermi

#### d-band center

In [18]:
help(vasptools.d_center)

Help on function d_center in module ageo.vasptools:

d_center(slab_xml, vac_pot_slab, fermi_slab)
    Calculate the d-band center of a structure
    
    Keyword arguments:
    slab_xml -- parsed vasprun.xml to be analysed.
    vac_pot_slab -- vacuum potential of the slab, or the Fermi energy of the
    slab.
    fermi_slab -- fermi energy extracted from vasprun.xml.



In [19]:
Ni_atom_d_center = vasptools.d_center(Ni_atom_xml, Ni_atom_Efermi, Ni_atom_Efermi)

Ni_bulk_d_center = vasptools.d_center(Ni_bulk_xml, Ni_bulk_Efermi, Ni_bulk_Efermi)

Ni_slab_100_d_center = vasptools.d_center(Ni_slab_100_xml, Ni_slab_100_Efermi, Ni_slab_100_Efermi)

### Adding the results into the database

In [20]:
#Grouping the data for each id
Ni_atom_list = {"e_fermi" : Ni_atom_Efermi,
                "d_center" : Ni_atom_d_center}

Ni_bulk_list = {"e_fermi" : Ni_bulk_Efermi,
                "d_center" : Ni_bulk_d_center,
                "e_cohesive" : Ni_bulk_e_coh}

Ni_slab_100_list = {"e_fermi" : Ni_slab_100_Efermi,
                    "d_center" : Ni_slab_100_d_center,
                    "e_surf_form" : Ni_slab_100_Ef}

In [21]:
db = db.connect('project.db')

In [22]:
db.update(1, e_fermi=Ni_atom_Efermi)
db.update(1, d_center=Ni_atom_d_center)

db.update(2, e_fermi=Ni_bulk_Efermi)
db.update(2, d_center=Ni_bulk_d_center)
db.update(2, e_cohesive=Ni_bulk_e_coh)

db.update(3, e_fermi=Ni_slab_100_Efermi)
db.update(3, d_center=Ni_slab_100_d_center)
db.update(3, e_surf_form=Ni_slab_100_Ef)

(1, 0)

### Showing the final database

In [23]:
!ase db 'project.db' -c id,formula,energy,d_center,e_fermi,e_cohesive,e_surf_form

id|formula|  energy|d_center|e_fermi|e_cohesive|e_surf_form
 1|Ni     |  -0.545|  -1.045| -4.617|          |           
 2|Ni4    | -21.883|  -1.982|  9.412|    -4.926|           
 3|Ni112  |-585.447|  -1.884| -1.371|          |      0.138
Rows: 3
Keys: d_center, e_cohesive, e_fermi, e_surf_form, structure_path
